# Annotated 3D structures (using modified PDB & ChimeraX)


Package requirements:
---------------------
Python 3  
IOData  
Numpy  


Goal:
-----
Make a ChimeraX script to visualize molecules with annotations (e.g. IOData attributes).


The process is as follows:
1. Use a PDB file format to represent the molecule (fname.pdb) and write the scalar property to be annotated as the `B` factor field (e.g. charges). Since IOData supports PDB files this is an easy task.  
2. Save this coordinates file.  
2. Write a ChimeraX script that loads the fname.pdb and labels the atoms by the defined `B` factors.

In [7]:
#
# Import required modules
#
from os import path
import numpy as np
from iodata import load_one, dump_one

Bellow some auxiliar functions are defined to handle finding the initial molecule file and writing the PDB and ChimeraX scripts.

In [8]:
def find_datafile(fname):
    """Path finder helper function """
    # Construct absolute path and split by package name.
    # root = path.abspath(__file__).split("chemtools")[0]
    # See stackoverflow solutions for `__file__ does not exist in Jupyter Notebook` 
    root = os.getcwd().split("chemtools")[0]
    DATAPATH = path.join(root, "chemtools", "chemtools", "data", "examples", fname)
    return path.abspath(DATAPATH)


def pdb_assign_bfactors(fname, mol, values):
    """Write given values as PDB's `B` factor field and dump file.

    Parameters
    ----------
    fname : str
        Name of the output PDB file.
    mol : iodata.iodata.IOData
        `IOData` instance from `iodata` module.
    values : list/np.ndarray
        A (N,) sequence of float values to be annotated.

    Raises
    ------
    ValueError
        If mol is not an IOData instance
    TypeError
        If values is not a 1D-array with N atoms elements.
    """
    if not (mol.__class__.__name__ == "IOData"):
        raise ValueError("`mol` must be an IOData instance.")
    if not (isinstance(values, np.ndarray) and len(values) == mol.natom and len(values.shape) == 1):
        raise TypeError("`values` must be a 1D-array of length N atoms.")
    mol.extra["bfactors"] = values
    dump_one(data, f"{fname}.pdb")


def print_cx_script_annotations(scriptfile, pdbfile):
    """Write CX (ChimeraX) script with annotated structure according to 
    `B` factors in PDB file

    Parameters
    ----------
    scriptfile : str
        Name of CX script file to generate.
    pdbfile : str
        Name of .pdb file used in CX script to visualize the annotations.
    """
    output = (
        f"open {pdbfile}.pdb\n"
        "# Select molecule by ID and annotate bfactor.\n"
        "# By default Model ID 1 is assigned.\n"
        "label #1 atoms attribute bfactor\n"
    )

    with open(scriptfile, "w") as f:
        f.write(output)

Steps 1. and 2.: Generate and store the PDB
-------------------------------------------

In [9]:
# 1. Load molecule data
fname = "dichloropyridine26_q+0"
data = load_one(find_datafile(f"{fname}.fchk"))

# 2. Generate PDB file (fname.pdb) with scalar property as `b` factors.
#    In this example the ESP charges are used.
esp = data.atcharges["esp"]
pdb_assign_bfactors(fname, data, esp)


Steps 3.: Write the ChimeraX script
------------------------------------

In [10]:

# 3. Generate ChimeraX script: fname.cxc
#    To visualize the molecule, use command: $ chimerax fname.cxc
scriptfile = fname + ".cxc"
print_cx_script_annotations(scriptfile, fname)